
# BrainCell 快速建模教程


## 一、基础元件建模
基础元件是构建神经元模型的核心单元，包含 **离子（Ion）** 和 **离子通道（IonChannel）**，直接决定神经元电生理特性（如动作电位产生、膜电流变化）。


### 1.1 离子建模
**作用**：
离子（如 Na⁺、K⁺、Ca²⁺）是神经元电活动的核心载体，其跨膜流动产生电流，驱动膜电位变化。
**核心功能**：
- 定义离子基本属性：名称、大小、通道、ion concertration、reversal potential。
- 支持与离子通道关联，建立通道与离子的选择性通透关系（如 Na⁺ 通道仅允许 Na⁺ 通过）。


### 1.2 离子通道建模
**作用**：
模拟细胞膜上选择性通透离子的蛋白结构，控制离子流动。
**核心功能**：
- 定义通道特性：关联离子类型、最大电导、门控动力学。
- 支持 Hodgkin-Huxley 型动力学。


## 二、房室建模
房室是模拟神经元空间结构的基本单元，可独立或组合使用，实现从单区域到全神经元的动态模拟。


### 2.1 单房室
**作用**：
简化模拟神经元局部区域，快速验证离子通道组合的电生理特性。
**核心功能**：
- 定义隔室物理参数：名称、大小、膜电容、阈值电压。
- 支持添加离子通道，构建“膜-通道”耦合系统，计算跨膜电流和膜电位变化。


### 2.2 多房室
**作用**：
模拟神经元复杂空间结构，实现了具有多个连接隔室的详细神经元模型，研究信号在神经元内的空间传播。
**核心功能**：
- 构建树状/链状房室网络：通过 `parent` 参数指定房室连接关系。
- 支持参数设置：不同隔室可定义独立的几何尺寸、电学特征，模拟真实神经元的形态异质性。


### 2.3 HH 模型建模
**作用**：
基于经典 Hodgkin-Huxley 理论的单房室模型，建模动作电位产生的完整过程。
**核心优势**：
- 开箱即用：
- 可扩展性：支持修改相关参数，用于验证不同离子机制对神经元兴奋性的影响。


## 三、微分方程建模
通过 **DiffEqState** 和 **DiffEqModule** 实现神经元动态的数学描述，将生理过程转化为状态变量演化的微分方程系统。


### 3.1 DiffEqState：状态变量管理
**作用**：
统一管理系统状态变量（如膜电位 `v`、通道门控变量 `m/h/n`），提供标准化的初始化、存储和访问接口。
**核心功能**：
- 定义多维度状态变量：支持常微分方程和随机微分方程，适配单神经元或网络模拟。
- 兼容 JAX 数组：无缝对接 JAX 自动微分和向量化计算，提升大规模模拟效率。


### 3.2 DiffEqModule：微分方程定义
**作用**：
定义状态变量的演化规则（即微分方程 `dx/dt = f(x, t)`），描述膜电位、门控变量随时间的变化规律。
**核心功能**：
- 实现梯度方法：输入当前状态和时间，返回各变量的导数（如 `dv/dt` 表示膜电位变化率）。
- 支持集成后处理：


## 快速上手建议
1. **从单房室开始**：使用 `HHNeuron` 类快速复现动作电位，熟悉基础工作流。
2. **模块化扩展**：逐步添加自定义离子通道或多房室结构，利用 BrainCell 的组件化设计降低复杂度。
3. **结合 JAX 加速**：通过 `jit` 优化模拟性能，尤其适用于大规模网络或参数扫描场景。

通过分层建模框架，``BrainCell`` 支持从分子机制（离子通道）到系统动态（神经元网络）的全尺度模拟，兼顾生物物理精确性与计算效率。